Research that studies depth of main and attention branches in unet_att_params model, that reconstructs noise in main branch

In [1]:
import sys
sys.path.append('../..')

import warnings
warnings.filterwarnings('ignore')
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import shutil

import datetime
import pprint

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


from seismicpro.batchflow import Pipeline, C, V, B, L, I, W

from seismicpro.batchflow.batchflow.models.tf import UNet, VGG7, VGG16, VGG19, ResNet, VGG
from seismicpro.batchflow.batchflow.models.tf.nn.activations import h_sigmoid

from seismicpro.batchflow.batchflow.research import Research, Option, KV, Results
from seismicpro.batchflow.batchflow.utils import plot_results_by_config

from seismicpro.src import (SeismicDataset, FieldIndex, TraceIndex, statistics_plot,
                            seismic_plot, spectrum_plot, merge_segy_files)
from seismicpro.models import attention_loss_gauss, attention_loss, UnetAtt
from seismicpro.models.unetatt_3 import UnetAttention4, UnetAttention3


from Ground_Roll.src.unet_att2_params import UnetAttParams

%env CUDA_VISIBLE_DEVICES=3
GPUs=[3]

env: CUDA_VISIBLE_DEVICES=3


In [2]:
path_raw = '/notebooks/data/H1_WZ/NA/1_NA-gr_input_DN01.sgy',
path_lift = '/notebooks/data/H1_WZ/NA/1_NA-gr_output_DN03.sgy',

index = (FieldIndex(name='raw', extra_headers=['offset'], path=path_raw)
         .merge(FieldIndex(name='lift', path=path_lift)))
index.head()

,TraceNumber,offset,TRACE_SEQUENCE_FILE,file_id,TRACE_SEQUENCE_FILE,file_id
,,,raw,raw,lift,lift
FieldRecord,,,,,,
111906,1512,326,1,/notebooks/data/H1_WZ/NA/1_NA-gr_input_DN01.sgy,1,/notebooks/data/H1_WZ/NA/1_NA-gr_output_DN03.sgy
111906,1513,326,2,/notebooks/data/H1_WZ/NA/1_NA-gr_input_DN01.sgy,2,/notebooks/data/H1_WZ/NA/1_NA-gr_output_DN03.sgy
111906,1511,333,3,/notebooks/data/H1_WZ/NA/1_NA-gr_input_DN01.sgy,3,/notebooks/data/H1_WZ/NA/1_NA-gr_output_DN03.sgy
111906,1514,334,4,/notebooks/data/H1_WZ/NA/1_NA-gr_input_DN01.sgy,4,/notebooks/data/H1_WZ/NA/1_NA-gr_output_DN03.sgy
111906,1515,348,5,/notebooks/data/H1_WZ/NA/1_NA-gr_input_DN01.sgy,5,/notebooks/data/H1_WZ/NA/1_NA-gr_output_DN03.sgy


In [3]:
print("Number of fields:", len(index))

Number of fields: 101


In [19]:
tri = np.random.choice(len(index.indices), size=60, replace=False)
train_indices = index.indices[tri]

print("Fields in CV:", sorted(train_indices))
print("Fields not in CV:", sorted(set(index.indices) - set(train_indices)))

Fields in CV: [111906, 111907, 111910, 111912, 111914, 111919, 111920, 111921, 111922, 111923, 111925, 111930, 111932, 111933, 111934, 111935, 111937, 111938, 111940, 111942, 111943, 111944, 111946, 111948, 111950, 111952, 111953, 111954, 111956, 111958, 111960, 111961, 111962, 111963, 111964, 111966, 111968, 111971, 111975, 111976, 111978, 111979, 111980, 111984, 111985, 111986, 111991, 111992, 111993, 111994, 111995, 111996, 111999, 112000, 112003, 112004, 112006, 112007, 112008, 112009]
Fields not in CV: [111908, 111909, 111911, 111913, 111915, 111916, 111917, 111918, 111924, 111927, 111928, 111929, 111931, 111936, 111939, 111941, 111945, 111951, 111955, 111957, 111959, 111965, 111967, 111969, 111970, 111972, 111973, 111974, 111977, 111981, 111982, 111983, 111987, 111988, 111989, 111990, 111997, 111998, 112001, 112002, 112005]


In [20]:
tindex = TraceIndex(index.create_subset(train_indices))
train_set = SeismicDataset(tindex)
print("Number of traces in train set:", len(tindex))

Number of traces in train set: 95616


In [5]:
common_config = {
    'initial_block/inputs': 'trace_raw',
    'inputs': dict(trace_raw={'shape': (3000, 1)},
                   lift={'name': 'targets', 'shape': (3000, 1)}),
    'optimizer': ('Adam', {'learning_rate': 0.0001}),
    'common/data_format': 'channels_last',
}

main_bl_config = {
    'encoder/num_stages': 4,
    'encoder/blocks': dict(layout='caca', filters=[16, 32, 64, 128], kernel_size=7, activation=tf.nn.elu),
    'encoder/downsample': dict(layout='pd', pool_size=2, pool_strides=2, dropout_rate=0.05),
    'encoder/order': ['block', 'skip', 'downsampling'],
    'embedding': dict(layout='caca', kernel_size=7, filters=256),
    'decoder/num_stages': 4,
    'decoder/blocks': dict(layout='caca', filters=[16, 32, 64, 128][::-1],
                           kernel_size=7, activation=tf.nn.elu),
    'decoder/upsample': dict(layout='tad', kernel_size=7, strides=2,
                             filters=[16, 32, 64, 128][::-1],
                             dropout_rate=0.05, activation=tf.nn.elu,),
    'decoder/order': ['upsampling', 'combine', 'block'],
}

unetatt_old_bl_config = {
    'loss': (attention_loss, {'balance': 0.05}),
    'body': {'main': main_bl_config,
    'attn': {
                'encoder/num_stages': 3,
                'encoder/blocks': dict(layout='caca', filters=[8, 16, 32], kernel_size=3, activation=tf.nn.elu),
                'encoder/downsample': dict(layout='pd', pool_size=2, pool_strides=2, dropout_rate=0.05),
                'embedding': dict(layout='caca', filters=64, kernel_size=3, activation=tf.nn.elu),
                'decoder/num_stages': 3,
                'decoder/blocks': dict(layout='caca', filters=[8, 16, 32][::-1], kernel_size=3, activation=tf.nn.elu),
                'decoder/upsample': dict(layout='tad', filters=[8, 16, 32][::-1], kernel_size=3, strides=2,
                                 dropout_rate=0.05, activation=tf.nn.elu),
            },
    },
    'train_steps': {
        'step_main': {'scope': 'main_branch'},
        'step_attention': {'scope': ['attention_branch', 'attention_dense']},
    }
}

main_config_small = {
    'encoder/num_stages': 4,
    'encoder/blocks': dict(layout='caca', 
                           filters=[16, 32, 64, 128], 
                           kernel_size=[7, 5, 5, 5], 
                           activation=tf.nn.elu),
    'encoder/downsample': dict(layout='pd', pool_size=2, pool_strides=2, dropout_rate=0.05),
    'encoder/order': ['block', 'skip', 'downsampling'],
    'embedding': dict(layout='ca ca', kernel_size=5, filters=256),
    'decoder/num_stages': 4,
    'decoder/blocks': dict(layout='caca', 
                           filters=[16, 32, 64, 128][::-1],
                           kernel_size=[7, 5, 5, 5][::-1], 
                           activation=tf.nn.elu),
    'decoder/upsample': dict(layout='tad',
                             filters=[16, 32, 64, 128][::-1],
                             kernel_size=[7, 5, 5, 5][::-1], 
                             strides=2, dropout_rate=0.05, activation=tf.nn.elu,),
    'decoder/order': ['upsampling', 'combine', 'block'],
}

unetatt_new_small_config = {
    'loss': (attention_loss, {'balance': 0.05}),
    'body' : main_config_small,
    'body/attention': {'blocks': dict(layout='caca', filters=[128, 64, 32, 16], kernel_size=5, activation=tf.nn.elu),
                       'upsample': dict(layout='tad', filters=[128, 64, 32, 16], kernel_size=5, strides=2,
                                        dropout_rate=0.05, activation=tf.nn.elu)},
}

unetatt2_new_small_config = {
    **unetatt_new_small_config,
    'body/attention/skip': True
}

unetatt_new_small_for4_config = {
    **unetatt_new_small_config,
    'body/attention': {'blocks': dict(layout='caca', filters=[128, 64, 32, 16], kernel_size=[3, 5, 5, 5], activation=tf.nn.elu),
                       'upsample': dict(layout='tad', filters=[128, 64, 32, 16], kernel_size=[3, 5, 5, 5], strides=2,
                                        dropout_rate=0.05, activation=tf.nn.elu)},
}

m1_vgg_small = {
    'loss': (attention_loss_gauss, {'balance': 0.05}),
    'common/main_base_class': UNet,
    'common/att_base_class': VGG,
    'body': {
        'main': main_config_small,
        'att': {'arch': [
                                (3, 0, 16, 1),
                                (3, 0, 32, 1),
                                (3, 0, 64, 1),
                                (3, 0, 128, 1),
                                (2, 0, 256, 1),
                            ]
            },
    },
    'head': {
        'main': dict(layout='c', filters=1, units=1),
        'att': dict(layout='fa', units=2, activation=h_sigmoid),
    },
    
    'train_steps': {'step_main': {'scope': 'main_branch'},
                 'step_attention': {'scope': 'attention_branch'}}
}

m1_resnet_small = {
    **m1_vgg_small,
    'common/att_base_class': ResNet,
    'body/att': {'num_blocks': [2, 2, 2, 2], 'filters': [16, 32, 64, 128], 'kernel_size': 5},
}

m3_vgg = {
    **m1_vgg_small,
    'head/mode': 'noise'
}

m3_resnet = {
    **m1_resnet_small,
    'head/mode': 'noise'
}

In [6]:
items = (
    ('egor_baseline', UnetAtt, {**common_config, **unetatt_old_bl_config}, 'loss_step_main'),
    ('unetatt2_small', UnetAttention3, {**common_config, **unetatt2_new_small_config}, 'loss'),
    ('unetatt3_small', UnetAttention3, {**common_config, **unetatt_new_small_config}, 'loss'),
    ('unetatt4_small', UnetAttention4, {**common_config, **unetatt_new_small_for4_config}, 'loss'),
    ('m1_vgg_small', UnetAttParams, {**common_config, **m1_vgg_small}, 'loss_step_main'),  
    ('m1_resnet_small', UnetAttParams, {**common_config, **m1_resnet_small}, 'loss_step_main'),
    ('m3_vgg_small', UnetAttParams, {**common_config, **m3_vgg}, 'loss_step_main'),  
    ('m3_resnet_small', UnetAttParams, {**common_config, **m3_resnet}, 'loss_step_main'),
)

keys, models, configs, losses = tuple(zip(*items))

grid = Option.product(Option('model', models), 
                      Option('model_config', [KV(c, alias=k) for k, c in zip(keys, configs)]),
                      Option('loss_name', losses))
print(list(grid.gen_configs()))

[ConfigAlias({'model': 'UnetAtt', 'loss_name': 'loss_step_main', 'model_config': 'egor_baseline'}), ConfigAlias({'model': 'UnetAttention3', 'loss_name': 'loss', 'model_config': 'unetatt2_small'}), ConfigAlias({'model': 'UnetAttention3', 'loss_name': 'loss', 'model_config': 'unetatt3_small'}), ConfigAlias({'model': 'UnetAttention4', 'loss_name': 'loss', 'model_config': 'unetatt4_small'}), ConfigAlias({'model': 'UnetAttParams', 'loss_name': 'loss_step_main', 'model_config': 'm1_vgg_small'}), ConfigAlias({'model': 'UnetAttParams', 'loss_name': 'loss_step_main', 'model_config': 'm1_resnet_small'}), ConfigAlias({'model': 'UnetAttParams', 'loss_name': 'loss_step_main', 'model_config': 'm3_vgg_small'}), ConfigAlias({'model': 'UnetAttParams', 'loss_name': 'loss_step_main', 'model_config': 'm3_resnet_small'})]


For reference: number of trainable parameters in models:


| model | trainable params|
|-------|-----------------|
|egor_baseline| 1 721 117|
|unetatt2_small| 1 747 794|
|unetatt3_small| 1 747 794|
|unetatt4_small| 1 693 266|
|m1_vgg_small| 1 711 395|
|m1_resnet_small| 1 691 955|
|m3_vgg_small| 1 711 395|
|m3_resnet_small| 1 691 955|

In [7]:
BATCH_SIZE=64
ITERATIONS=5000 # ~ 5 epochs of train in CV
TEST_EXECUTE_FREQ=200

In [8]:
def exp_stack(x):
    return np.expand_dims(np.vstack(x), -1)

def make_data(batch, **kwagrs):
    return {'trace_raw': exp_stack(batch.raw), 'lift': exp_stack(batch.lift)}

def make_data_inference(batch, **kwagrs):
    return {'trace_raw': exp_stack(batch.raw)}


train_template = (Pipeline()
                  .load(components=('raw', 'lift'), fmt='segy', tslice=np.arange(3000))
                  .init_model('dynamic', C('model'), name='unet', config=C('model_config'))
                  .init_variable('train_loss')
                  .train_model('unet', make_data=make_data, fetches=C('loss_name'),
                               save_to=V('train_loss'))
                  .run_later(BATCH_SIZE, shuffle=True, n_epochs=None)
                 )

test_template = (Pipeline()
                 .init_variable('test_loss')
                 .load(components=('raw', 'lift'), fmt='segy', tslice=np.arange(3000))
                 .import_model('unet', C('import_from'))
                 .predict_model('unet', make_data=make_data, fetches=C('loss_name'),
                                save_to=V('test_loss'))
                 .run_later(BATCH_SIZE, shuffle=True, n_epochs=1))


In [9]:
def dump_results(iteration, experiment, train_ppl, val_set):
    
    tmp_dump_path = os.path.join(experiment[train_ppl].path, 'tmp')
    if os.path.exists(tmp_dump_path):
        shutil.rmtree(tmp_dump_path)
    os.makedirs(tmp_dump_path)
    
    inference_pipeline = (val_set.p
                         .load(components='raw', fmt='segy', tslice=np.arange(3000))
                         .import_model('unet', experiment[train_ppl].pipeline)
                         .predict_model('unet', make_data=make_data_inference, fetches=["out_lift"], save_to=B('raw'))
                         .dump(path=L(lambda x: os.path.join(tmp_dump_path, str(x) + '.sgy'))(I()), src='raw', fmt='segy', split=False)
                         )

    inference_pipeline.run(1000, n_epochs=1, drop_last=False, shuffle=False, bar=False)
    
    output_path = os.path.join(experiment[train_ppl].path, 'res.sgy')
    merge_segy_files(output_path=output_path, extra_headers='all', path=os.path.join(tmp_dump_path, '*.sgy'), bar=False)

    if os.path.exists(tmp_dump_path):
        shutil.rmtree(tmp_dump_path) 
    
    return output_path


In [21]:
val_index = TraceIndex(index)
val_set = SeismicDataset(val_index)
print("Number of traces in val set:", len(val_index))

Number of traces in val set: 160416


In [10]:
!rm -r 7_research_res

In [11]:
res_name = '7_research_res'

val_set = TraceIndex(index)

research = (Research()
            .add_pipeline(train_template, dataset=train_set, part='train',
                          variables='train_loss', name='train_ppl', logging=True)
            .add_pipeline(test_template, dataset=train_set, part='test', variables='test_loss', name='test_ppl',
                          execute=TEST_EXECUTE_FREQ, run=True, import_from='train_ppl')
            .add_function(dump_results, returns='output_path', name='dump_res_fn',
                          execute='last', train_ppl='train_ppl', val_set=val_set)
            .add_grid(grid)
            )

research.run(n_iters=ITERATIONS, n_splits=3, shuffle=True, name=res_name, bar=True, gpu=GPUs)


Research 7_research_res is starting...


  0%|          | 0/48 [00:00<?, ?it/s]

Distributor has 24 jobs with 2 iterations. Totally: 48


100%|██████████| 48/48 [15:24<00:00, 19.27s/it]


In [12]:
res = research.load_results(use_alias=True)

In [13]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 0
Data columns (total 8 columns):
config         96 non-null object
cv_split       96 non-null int64
iteration      96 non-null int64
name           96 non-null object
output_path    24 non-null object
repetition     96 non-null int64
test_loss      24 non-null float64
train_loss     48 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.8+ KB


In [ ]:
def plot_results_by_config(results, variables, figsize=None, force_flat=True):
    if isinstance(variables, dict):
        variables = variables.items()
    elif len(variables) == 2 and isinstance(variables[0], str):
        variables = (variables,)

    gbc = results.groupby('config')
    n_configs = len(gbc)
    n_vars = len(variables)

    n_h, n_v = (n_configs, 1) if n_vars == 1 and force_flat else (n_vars, n_configs)

    if figsize is None:
        figsize = (n_h * 5, n_v * 5)

    _, axs = plt.subplots(n_v, n_h, figsize=figsize)
    for x, (config, df) in enumerate(gbc):
        for y, (source, val) in enumerate(variables):
            if n_configs == 1 and n_vars == 1:
                ax = axs
            elif n_configs == 1:
                ax = axs[y]
            elif n_vars == 1:
                ax = axs[x]
            else:
                ax = axs[x, y]
            
            col = 'cv_split' if 'cv_split' in df.columns else 'repetition'

            (df[df['name'] == source]
             .pivot(index='iteration', columns=col, values=val)
             .rename(columns=lambda s: col + str(s))
             .plot(ax=ax))
            ax.set_title(config)
            ax.set_xlabel('Iteration')
            ax.set_ylabel(val.replace('_', ' ').capitalize())
            ax.grid(True)
            ax.legend()

In [ ]:
plot_results_by_config(res, {'train_ppl': 'train_loss', 'test_ppl': 'test_loss'})

In [ ]:
res2 = research.load_results()
res2.head()

In [ ]:
(res2[res2.iteration >= 4000]
 .groupby('model_config')['train_loss', 'test_loss']
 .agg(['mean', 'std'])
 .swaplevel(axis=1)
 .style.background_gradient(axis=None, cmap='cool', subset=['mean'])
 .background_gradient(axis=None, cmap='cool', subset=['std'])
)

